In [35]:
from datetime import datetime
import json
import urllib
import urllib2
import re

from bs4 import BeautifulSoup



hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}



def getUrlSoup(url,hdr):
    req1 = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req1)
    content = page.read()
    soup = BeautifulSoup(content)
    return soup



def getReviews(main_block, reviews_obj=None):
    
    assert reviews_obj is not None
    
    # Treat it as a massive block of text and split it on the horizontal lines.
    reviews_block = re.split(r"<hr[^>]*>", str(main_block))
    
    # Iterate over all the reviews.
    # NOTE: The indexing is dubious here...
    for r in reviews_block[1:-3]:
        
        # Soup the block of html-code-text
        soup_review = BeautifulSoup(r)
        soup_review_header = soup_review.find('div')
        
        # Create an empty dictionary to hold the review.
        review = dict()
        
        # Get the user-name, user-link for the reviewer.
        review['user_name'] = soup_review_header.findAll('a')[1].get_text()
        review['user_href'] = soup_review_header.findAll('a')[1]['href']

        # Try to get the rating.
        soup_imgs = soup_review_header.findAll('img')
        review['review_rating'] = None
        if len(soup_imgs) > 1:
            review['review_rating'] = int(soup_imgs[1]['alt'].split('/')[0])
        
        # Get the review title.
        review['review_title'] = soup_review.find('h2').get_text()
        
        # Get the review content.
        tmp_content = soup_review.findAll('p')
        if len(tmp_content) > 1 and 'spoilers' in tmp_content[0].get_text():
            review['review_content'] = tmp_content[1].get_text()
        else:
            review['review_content'] = tmp_content[0].get_text()
        
        # Get the review usefulness.
        # review['review_useful'] = soup_review_header.find('small').get_text().split('people')[0].rstrip().lstrip()
        
        reviews_obj.append(review)
        
    # Done. Return the reviews_obj
    return reviews_obj

Load the list of movies.

In [36]:
# MOVIES = json.load(open('../../list_of_movies.json','r'))
MOVIES = json.load(open('../../list_of_movies_with_reviews.json','r'))

print 'We have {} movies in the dataset.\n'.format(len(MOVIES))
# print json.dumps(MOVIES[2],indent=4)

We have 10000 movies in the dataset.



For each movie we will grab the ID and get the reviews.

In [41]:
print 'Grabbing reviews for all movies...'

# Iterate over all movies.
for e,movie in enumerate(MOVIES[:50]):
    
    # Check if the movie already has the reviews populated
    if 'reviews' not in movie:
        
        print '\n\t\t ---> ', movie['name'],
    
        url = 'http://www.imdb.com' + movie['link'] + 'reviews'

        # Get the contents for the first reviews page.
        soup = getUrlSoup(url,hdr)
        # Get the main content block.
        main_block = (soup.body.find('div', attrs={'class':'reviews'})
                               .find('div', attrs={'id':'tn15main'})
                               .find('div', attrs={'id':'tn15content'}))
        # Get the number of pages to get reviews from.
        num_pages = int(main_block.findAll('table')[1]
                                  .find('td',attrs={'nowrap':'1'}).get_text().split(' of ')[1].rstrip(':'))

        MOVIES[e]['num_reviews_pages'] = num_pages
        print ' -- pages: ', num_pages

        # Iterate over all the reviews pages.
        REVIEWS = []

        for i in range(1,num_pages+1):

            # If this is the first page, we don't need to load the new page.
            if i == 1:
                REVIEWS = getReviews(main_block,REVIEWS)

            # We need to grab the contents for the next page.
            else:
                page_soup = getUrlSoup(url + '?start=' + str(10*(i-1)), hdr)
                main_block = (page_soup.body.find('div', attrs={'class':'reviews'})
                                            .find('div', attrs={'id':'tn15main'})
                                            .find('div', attrs={'id':'tn15content'}))
                REVIEWS = getReviews(main_block,REVIEWS)

        # Done for this movie. Add reviews to the movie object.
        MOVIES[e]['reviews'] = REVIEWS

    
    if e % 1 == 0:
        print e, '.',

Grabbing reviews for all movies...
0 . 1 . 2 . 3 . 4 . 5 . 6 . 7 . 8 . 9 . 10 . 11 . 12 . 13 . 14 . 15 . 16 . 17 . 18 . 19 . 20 . 21 . 22 . 23 . 24 . 25 . 26 . 27 . 28 . 29 . 
		 --->  The Departed  -- pages:  203


KeyboardInterrupt: 

In [33]:
# print json.dumps(MOVIES[2],indent=4)

Save the data to a new file.

In [40]:
# Full dump
with open('../../list_of_movies_with_reviews.json', 'w') as outfile:
    json.dump(MOVIES, outfile)

In [ ]:
# # Dump JSONL line by line
# with open('../../list_of_movies_with_reviews.json', 'a') as the_file:
# the_file.write('Hello\n')